# Format Data

## Preprocess raw text prior to modeling

Note: my initial approach involved using spaCy bigram and trigram modeling. I abandoned that approach because it was yielding terms like `java_python` or `docker_kubernetes` -- i.e., it was connecting skills that commonly appeared together.

That code exists in the notebook `modeling_lowercase.ipynb` in the `/sandbox` directory.

In [1]:
import os
import re
import json
from pprint import pprint
from bs4 import BeautifulSoup
from bs4.element import Comment
import sys

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import gensim
from pprint import pprint

from stop_words import get_stop_words

import codecs
import os
import time

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')

import spacy
import pandas as pd
import itertools as it
import en_core_web_sm

import spacy
nlp = spacy.load('en')

[nltk_data] Downloading package punkt to /home/goodgame/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/goodgame/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/goodgame/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import codecs
import boto3
# from collections import defaultdict
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [3]:
s3 = boto3.resource('s3')

In [4]:
BUCKET_NAME = 'tech-salary-project'

In [8]:
# %%time

# bucket = s3.Bucket(BUCKET_NAME)
# all_job_titles = []
# all_job_ids = []
# all_summaries = []
# for o in bucket.objects.all():
#     if o.key.startswith('salaries'):
#         continue
    
#     object = bucket.Object(o.key)
#     try:
#         lines = object.get()['Body'].read().decode('utf-8').splitlines()
#         for line in lines:
#             d = json.loads(line)
            
#             title = d['title']
#             jid = d['id']
#             summary = d['summary']
            
#             all_job_ids.append(jid)
#             all_job_titles.append(title)
#             all_summaries.append(summary)
#     except Exception as e:
#         continue

In [9]:
print(len(set(all_summaries)))

119326


In [3]:
%%time

# Put dataset in memory
directory = os.fsencode('../local_data/')

original_texts = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    full_file = '../local_data/' + str(filename)
    with open(full_file, 'r') as infile:
        text = infile.read()
        original_texts.append(text)

CPU times: user 17.4 s, sys: 13.2 s, total: 30.6 s
Wall time: 26min 57s


In [22]:
%%time
# Load in Stackoverflow data

stackoverflow_raw = []

def clean_text(text):
    cleanr = re.compile(r'\bamp\b')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

with open('../stackoverflow_data/stack_parsed.txt', 'r') as infile:
    for line in infile:
        line = line.strip()
        line = clean_text(line)
        if line == '':
            continue
        stackoverflow_raw.append(line)

# Size of list in bytes
print(sys.getsizeof(stackoverflow_raw))
print(len(stackoverflow_raw))

4290016
519155
CPU times: user 11.7 s, sys: 325 ms, total: 12 s
Wall time: 14.9 s


In [8]:
print("Length of job data list:", len(original_texts))
print("Length of Stackoverflow data list:", len(stackoverflow_raw))
all_data = original_texts + stackoverflow_raw
print("Combined dataset length:", len(all_data))

print("Size of total dataset in MB:", sys.getsizeof(all_data)/1000000)

Length of job data list: 106373
Length of Stackoverflow data list: 519155
Combined dataset length: 625528
Size of total dataset in MB: 5.004288


In [23]:
%%time
lower_stackoverflow = [item.lower().strip() for item in stackoverflow_raw]

CPU times: user 404 ms, sys: 236 ms, total: 640 ms
Wall time: 638 ms


## Manually replace bigrams.

In [26]:
# subset = lower_data[::10000]
# print("Items in subset:",len(subset))

Items in subset: 63


In [26]:
# Load pickled skill dictionary from disk. I generated it from prior work.
with open('skill_bigram_dict.pkl', 'rb') as handle:
    skill_dict = pickle.load(handle)
print("machine learning becomes", skill_dict['machine learning'])

machine learning becomes machine_learning


In [24]:
def multiple_replace(dict, text):
    '''
    Function that uses regex to replace terms.
    Input is a dictionary of terms to switch, plus
    a string.
    '''
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text) 

In [27]:
%%time
# new_data = []
# for text in lower_data:
#     new_data.append(multiple_replace(skill_dict, text))

new_stackoverflow = []
for text in lower_stackoverflow:
    new_stackoverflow.append(multiple_replace(skill_dict, text))

CPU times: user 4min 56s, sys: 159 ms, total: 4min 56s
Wall time: 4min 56s


In [28]:
# Write list to file as a pickled object, to save time later
with open('../models_12apr/stackoverflow_data_skill_bigrams_list.pkl', 'wb') as f:
    pickle.dump(new_stackoverflow, f)

In [3]:
# Load back into memory
with open('../models_12apr/all_lowercase_data_skill_bigrams_list.pkl', 'rb') as handle:
    new_data = pickle.load(handle)

#### Note: All texts are lowercase at this point.

In [29]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    12APR: adding numbers to this
    """
    
    return token.is_punct or token.is_space or token.is_digit

def line_review(filename):
    """
    SRG: modified for a list
    generator function to read in text from the file
    and un-escape the original line breaks in the text
    """
    
    for text in filename:
        yield text.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_text in nlp.pipe(line_review(filename),
                                  batch_size=5000, n_threads=24):
        
        for sent in parsed_text.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [30]:
%%time

with codecs.open('../models_12apr/jds_skill_bigrams_concat_parsed_stackoverflow.txt', 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(new_stackoverflow):
        f.write(sentence + '\n')

CPU times: user 1d 1h 16min 47s, sys: 1d 13h 27min 12s, total: 2d 14h 44min
Wall time: 3h 40min 26s
